<a href="https://colab.research.google.com/github/Surya2000PratapSingh/Covid19/blob/main/Covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
# MODEL 1 MULTI CLASS CLASSIFIER

# Load the dataset
train_datagen = ImageDataGenerator(rescale=1./255)
train_ds = train_datagen.flow_from_directory(
    '/content/Covid19-dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_ds = test_datagen.flow_from_directory(
    '/content/Covid19-dataset/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')



# Define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.summary()

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history=model.fit(train_generator, epochs=10,validation_data=test_generator)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)

print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)




# Make predictions on the test set
test_predictions = model.predict(test_ds)

# Calculate accuracy, precision, recall, F1 score, confusion matrix, and AUROC
accuracy = accuracy_score(test_ds.classes, test_predictions.argmax(axis=1))
precision = precision_score(test_ds.classes, test_predictions.argmax(axis=1), average='micro')
recall = recall_score(test_ds.classes, test_predictions.argmax(axis=1), average='micro')
f1_score = f1_score(test_ds.classes, test_predictions.argmax(axis=1), average='micro')
confusion_matrix = confusion_matrix(test_ds.classes, test_predictions.argmax(axis=1))

# Print the results
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 score:', f1_score)
print('Confusion matrix:')
print(confusion_matrix)


Found 251 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [ ]:
# MODEL 2 (2 Level Classifier)

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


# Define paths for train and test datasets
train_data_dir = '/content/drive/MyDrive/Covid19-dataset/train'
test_data_dir = '/content/drive/MyDrive/Covid19-dataset/test'
img_width, img_height = 224, 224


# Rescale pixel values and perform data augmentation for the training dataset
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Only rescale pixel values for the testing dataset
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load and prepare the training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary'
)

# Load and prepare the testing data
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary'
)

# First Classifier: Normal vs. Abnormal
model_first = Sequential()
model_first.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation='relu'))
model_first.add(MaxPooling2D(pool_size=(2, 2)))
model_first.add(Flatten())
model_first.add(Dense(128, activation='relu'))
model_first.add(Dense(1, activation='sigmoid'))

# Compile the first model
model_first.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the first model
model_first.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

# Second Classifier: COVID vs. Pneumonia
abnormal_data_dir = '/content/drive/MyDrive/Covid19-dataset/train/Abnormal'
abnormal_generator = train_datagen.flow_from_directory(
    abnormal_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary'
)

model_second = Sequential()
model_second.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation='relu'))
model_second.add(MaxPooling2D(pool_size=(2, 2)))
model_second.add(Flatten())
model_second.add(Dense(128, activation='relu'))
model_second.add(Dense(1, activation='sigmoid'))

# Compile the second model
model_second.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the second model
model_second.fit_generator(
    abnormal_generator,
    steps_per_epoch=abnormal_generator.samples // abnormal_generator.batch_size,
    epochs=10,
)



# Final Evaluation
# Use model_first to predict if an image is normal or abnormal.
# If the image is abnormal, use model_second to predict whether it's COVID or pneumonia.


In [ ]:
test_loss1, test_accuracy1 = model_first.evaluate(test_generator)

print('Test loss:', test_loss1)
print('Test accuracy:', test_accuracy1)

test_loss,2 test_accuracy2 = model_second.evaluate(test_generator)

print('Test loss:', test_loss2)
print('Test accuracy:', test_accuracy2)